# XGBoost

In [1]:
# Функция для получения ввода от пользователя
def get_user_input():
    user_input = input("Введите 'all' для анализа всех продуктов или перечислите ID продуктов через запятую: ")
    if user_input.lower() == 'all':
        return None
    else:
        try:
            product_ids = list(map(int, user_input.split(',')))
            return product_ids
        except ValueError:
            print("Ошибка ввода. Пожалуйста, введите корректные ID.")
            return get_user_input()

In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import numpy as np
import pickle  # Import pickle for model saving and loading

# Load the Excel files
sales_data = pd.read_excel("C:/Users/ekart/OneDrive/Рабочий стол/Liudmila_data/sales (2).xlsx")
raw_material_data = pd.read_excel("C:/Users/ekart/OneDrive/Рабочий стол/Liudmila_data/raw_material_recipe.xlsx")
sales_stock = pd.read_excel("C:/Users/ekart/OneDrive/Рабочий стол/Liudmila_data/sales_stock.xlsx")
raw_material_stock = pd.read_excel("C:/Users/ekart/OneDrive/Рабочий стол/Liudmila_data/raw_material_stock.xlsx")

# Label encoding for categorical features
le = LabelEncoder()
sales_data['promo action'] = le.fit_transform(sales_data['promo action'])
sales_data['конкуренция'] = le.fit_transform(sales_data['конкуренция'])

# Selecting features for the model
features = sales_data[['price', 'cost price', 'рейтинг товаров', 'количество просмотров', 'конкуренция', 'promo action', 'seasonal_discount']]
target = sales_data['y']

# Handling missing values
imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_imputed, target, test_size=0.2, random_state=42)

# XGBoost Model
model = xgb.XGBRegressor(objective='reg:squarederror', enable_categorical=True, eval_metric="rmse")
param_grid = {'max_depth': [3, 4, 5, 7], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.8, 1], 'colsample_bytree': [0.8, 1], 'reg_alpha': [50, 100], 'reg_lambda': [40, 50, 100]}
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

# Save the model to a file
filename = 'finalized_model_xgb.pkl'
pickle.dump(best_model, open(filename, 'wb'))

# Load the model from the file
loaded_model = pickle.load(open(filename, 'rb'))

# Use the loaded model to make predictions (same as previous steps to prepare data)
# Assuming get_user_input() function is defined elsewhere and works correctly
user_choice = get_user_input()
if user_choice is None:
    top_products = sales_data['unique_id'].unique()
else:
    top_products = user_choice

next_week_features = sales_data[sales_data['unique_id'].isin(top_products)].groupby('unique_id').last()[list(features.columns)]
next_week_features = imputer.transform(next_week_features)
next_week_sales_predictions = loaded_model.predict(next_week_features)

predicted_sales = pd.DataFrame({
    'unique_id': top_products,
    'predicted_sales': np.round(next_week_sales_predictions)
})

# Проверка наличия товара на складе и расчет потребности в сырье
predicted_sales = predicted_sales.merge(sales_stock, on='unique_id', how='left')
predicted_sales['stock_shortage'] = predicted_sales['predicted_sales'] - predicted_sales['stock']
predicted_sales['production_needed'] = np.where(predicted_sales['stock_shortage'] <= 0, 0, predicted_sales['stock_shortage'])

# Расчет потребности в сырье
raw_material_needs = predicted_sales[predicted_sales['production_needed'] > 0].merge(raw_material_data, on='unique_id', how='left')
raw_material_needs = raw_material_needs.merge(raw_material_stock, on='raw_material_ID', how='left', suffixes=('_need', '_stock'))

# Перевод миллиграмм в граммы и расчет дефицита
raw_material_needs['volume_need_adjusted'] = np.where(raw_material_needs['unit_need'] == 'мг', raw_material_needs['production_needed'] * raw_material_needs['volume_need'] / 1000, raw_material_needs['production_needed'] * raw_material_needs['volume_need'])
raw_material_needs['volume_stock_adjusted'] = np.where(raw_material_needs['unit_stock'] == 'мг', raw_material_needs['volume_stock'] / 1000, raw_material_needs['volume_stock'])
raw_material_needs['raw_material_needed'] = (raw_material_needs['volume_need_adjusted'] - raw_material_needs['volume_stock_adjusted']).clip(lower=0)

# Обновление единиц измерения для материалов из мг в г
raw_material_needs['unit_final'] = np.where(raw_material_needs['unit_need'] == 'мг', 'г', raw_material_needs['unit_need'])

# Вывод результатов
results = raw_material_needs[['product name_x','unique_id', 'raw_material_ID', 'list_need', 'raw_material_needed', 'unit_final']]


results

Fitting 3 folds for each of 288 candidates, totalling 864 fits


,product name_x,unique_id,raw_material_ID,raw_material_needed,unit_final
0,"ALIVE Английская соль для ванн, 2 кг в упаковк...",9,192.0,125985.00,г
1,"ALIVE Английская соль для ванн, 2 кг в упаковк...",9,341.0,0.00,шт
2,"ALIVE Английская соль для ванн, 3 кг в упаковк...",11,192.0,176985.00,г
3,"ALIVE Английская соль для ванн, 3 кг в упаковк...",11,342.0,0.00,шт
4,"ALIVE Английская соль для ванн, 5 кг в упаковк...",12,69.0,392.00,шт
...,...,...,...,...,...
2396,Zапах _ Спрей для устранения неприятных запахо...,423,264.0,5191.16,г
2397,Zапах _ Спрей для устранения неприятных запахо...,423,336.0,2065.00,г
2398,Zапах _ Спрей для устранения неприятных запахо...,423,389.0,147848.00,г
2399,Zапах _ Спрей для устранения неприятных запахо...,423,704.0,80.00,шт


# LSTM

In [8]:
# Функция для получения ввода от пользователя
def get_user_input():
    user_input = input("Введите 'all' для анализа всех продуктов или перечислите ID продуктов через запятую: ")
    if user_input.lower() == 'all':
        return None
    else:
        try:
            product_ids = list(map(int, user_input.split(',')))
            return product_ids
        except ValueError:
            print("Ошибка ввода. Пожалуйста, введите корректные ID.")
            return get_user_input()



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Загрузка данных
sales_data = pd.read_excel("C:/Users/ekart/OneDrive/Рабочий стол/Liudmila_data/sales (2).xlsx")
raw_material_data = pd.read_excel("C:/Users/ekart/OneDrive/Рабочий стол/Liudmila_data/raw_material_recipe.xlsx")
sales_stock = pd.read_excel("C:/Users/ekart/OneDrive/Рабочий стол/Liudmila_data/sales_stock.xlsx")
raw_material_stock = pd.read_excel("C:/Users/ekart/OneDrive/Рабочий стол/Liudmila_data/raw_material_stock.xlsx")

# Подготовка данных
le = LabelEncoder()
sales_data['promo action'] = le.fit_transform(sales_data['promo action'])
sales_data['конкуренция'] = le.fit_transform(sales_data['конкуренция'])
sales_data['date'] = pd.to_datetime(sales_data['date'])
sales_data['year'] = sales_data['date'].dt.year
sales_data['month'] = sales_data['date'].dt.month
sales_data['day'] = sales_data['date'].dt.day
sales_data.drop('date', axis=1, inplace=True)

# Обработка пропущенных значений
imputer = SimpleImputer(strategy='mean')
features = ['price', 'cost price', 'рейтинг товаров', 'количество просмотров', 'конкуренция', 'promo action', 'seasonal_discount', 'year', 'month', 'day']
X = imputer.fit_transform(sales_data[features])
y = sales_data['y'].values

# Нормализация данных
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Преобразование данных в последовательности
sequence_length = 10
n_features = X.shape[1]
X_sequences = []
y_sequences = []
for i in range(sequence_length, len(X)):
    X_sequences.append(X[i-sequence_length:i, :])
    y_sequences.append(y[i])
X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# Обучение модели LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(sequence_length, n_features)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# Сохранение модели
model.save('lstm_model.h5')

In [ ]:
# Загрузка сохраненной модели
loaded_model = load_model('lstm_model.h5')

In [ ]:
# Получение выбора пользователя
user_choice = get_user_input()
if user_choice is None:
    top_products = sales_data['unique_id'].unique()
else:
    top_products = user_choice


# Подготовка данных для прогнозирования
next_week_features = sales_data[sales_data['unique_id'].isin(top_products)].groupby('unique_id').last()[features]
next_week_features = imputer.transform(next_week_features)
next_week_features = scaler_X.transform(next_week_features)

# Создание последовательностей
next_week_features = np.repeat(next_week_features[:, np.newaxis, :], 10, axis=1)

# Прогнозирование
next_week_sales_predictions = loaded_model.predict(next_week_features)

# Обратное преобразование масштабирования
predicted_sales = scaler_y.inverse_transform(next_week_sales_predictions.reshape(-1, 1))

# Создание DataFrame для вывода результатов
predicted_sales_df = pd.DataFrame({
    'unique_id': top_products,
    'predicted_sales': np.round(predicted_sales.flatten())
})

print(predicted_sales_df)

In [ ]:
# Проверка наличия товара на складе и расчет потребности в сырье
predicted_sales = predicted_sales_df.merge(sales_stock, on='unique_id', how='left')
predicted_sales['stock_shortage'] = predicted_sales['predicted_sales'] - predicted_sales['stock']
predicted_sales['production_needed'] = np.where(predicted_sales['stock_shortage'] <= 0, 0, predicted_sales['stock_shortage'])

# Расчет потребности в сырье
raw_material_needs = predicted_sales[predicted_sales['production_needed'] > 0].merge(raw_material_data, on='unique_id', how='left')
raw_material_needs = raw_material_needs.merge(raw_material_stock, on='raw_material_ID', how='left', suffixes=('_need', '_stock'))

# Перевод миллиграмм в граммы и расчет дефицита
raw_material_needs['volume_need_adjusted'] = np.where(raw_material_needs['unit_need'] == 'мг', raw_material_needs['production_needed'] * raw_material_needs['volume_need'] / 1000, raw_material_needs['production_needed'] * raw_material_needs['volume_need'])
raw_material_needs['volume_stock_adjusted'] = np.where(raw_material_needs['unit_stock'] == 'мг', raw_material_needs['volume_stock'] / 1000, raw_material_needs['volume_stock'])
raw_material_needs['raw_material_needed'] = (raw_material_needs['volume_need_adjusted'] - raw_material_needs['volume_stock_adjusted']).clip(lower=0)

# Обновление единиц измерения для материалов из мг в г
raw_material_needs['unit_final'] = np.where(raw_material_needs['unit_need'] == 'мг', 'г', raw_material_needs['unit_need'])

# Вывод результатов
results = raw_material_needs[['product name_x','unique_id', 'raw_material_ID', 'list_need', 'raw_material_needed', 'unit_final']]

results